# Bedrock Guardrails

Amazon Bedrock Guardrails are configurable safety and governance controls for generative AI applications, designed to filter harmful content and enforce responsible AI policies across prompts and responses. They help organizations standardize safety, privacy, and compliance when deploying large language models.

In [1]:
import boto3
import json

In [2]:
bedrock_runtime = boto3.client('bedrock-runtime')
model_id = "amazon.nova-micro-v1:0"

In [4]:
payload = {
    "messages":[{"role":"user","content":[{"text":"john is my friend"}]}],
    "inferenceConfig":{
        "maxTokens":50,
        "temperature":0.5,
        "topP":0.9
    }
}

- User prompt: "john is my friend"
- Guardrail check: Finds "john" in the input.
- Result: Prompt is blocked, and the model never processes it.

#### Apply Guardrails on invoke API

In [5]:
model_invoke = bedrock_runtime.invoke_model(modelId=model_id,
                                            guardrailIdentifier="xyehkzih9by1",
                                            guardrailVersion="DRAFT",
                                            body=json.dumps(payload))
response = json.loads(model_invoke['body'].read())
response

{'output': {'message': {'role': 'assistant',
   'content': [{'text': 'Sorry, the model cannot answer this question.'}]}},
 'usage': {},
 'amazon-bedrock-guardrailAction': 'INTERVENED'}

You will find the Guardrail Identifier and Version in the Bedrock dashboard. Watch my YouTube video for a walkthrough.

#### Apply Guardrails on Converse API

In [7]:
messages = [
    {"role":"user","content":[{"text":"what is ai and how its work"}]}
]
system = [{"text":"You are a helpful assistent."}]
inf_config = {
    "temperature":0.7, "maxTokens":50
}
guardrailConfig = {
    "guardrailIdentifier": "xyehkzih9by1",
    "guardrailVersion": "DRAFT"
}
response = bedrock_runtime.converse(modelId=model_id, 
                                    messages=messages,
                                    system=system,
                                    inferenceConfig=inf_config,
                                    guardrailConfig=guardrailConfig)
response

{'ResponseMetadata': {'RequestId': '9c5abb7c-6063-48aa-8dfd-4c10b4163b9c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 12 Jan 2026 11:03:57 GMT',
   'content-type': 'application/json',
   'content-length': '487',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9c5abb7c-6063-48aa-8dfd-4c10b4163b9c'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': "AI, or Artificial Intelligence, refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding. Here's a breakdown of how AI works and its various"}]}},
 'stopReason': 'max_tokens',
 'usage': {'inputTokens': 15, 'outputTokens': 50, 'totalTokens': 65},
 'metrics': {'latencyMs': 879}}